<a href="https://colab.research.google.com/github/WinsonTruong/hf-deeprl-course/blob/main/exploring_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyglet==1.5.1
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 12.5 MB/s 
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.5.0
    Uninstalling pyglet-1.5.0:
      Successfully uninstalled pyglet-1.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.17.3 requires pyglet<=1.5.0,>=1.4.0, but you have pyglet 1.5.1 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 496 kB of archives.
After

In [2]:
!pip install gym[box2d]
!pip install stable-baselines3[extra]
!pip install huggingface_sb3
!pip install pyglet
!pip install ale-py==0.7.4 # To overcome an issue with gym (https://github.com/DLR-RM/stable-baselines3/issues/875)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 12.2 MB/s 
     |████████████████████████████████| 448 kB 65.0 MB/s 
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.5.1
    Uninstalling pyglet-1.5.1:
      Successfully uninstalled pyglet-1.5.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 14.1 MB/s 
     |████████████████████████████████| 1.5 MB 59.1 MB/s 
     |████████████████████████████████| 1.6 MB 51.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616824 sha256=0b8ca8270ab3ff87f679892475cc51cce7886f657bcba250c7cb6b0595b749ea
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff

In [5]:
import gym

In [7]:
env = make_vec_env('LunarLander-v2', n_envs=32)
observation = env.reset()
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) 

Observation Space Shape (8,)
Sample observation [ 1.081068   -0.9088703  -0.8154723   0.640134    1.344351   -0.02062219
  0.81352013 -1.3294026 ]


In [33]:
## model1: ppo

model = PPO('MlpPolicy', env, verbose=1
            ,tensorboard_log = "./ppo_lunarlander/")
model.learn(total_timesteps = 500000, tb_log_name="run1")
model_name = "base ppo model"
model.save(model_name)

Using cuda device
Logging to ./ppo_lunarlander/run1_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 94.6     |
|    ep_rew_mean     | -186     |
| time/              |          |
|    fps             | 5069     |
|    iterations      | 1        |
|    time_elapsed    | 12       |
|    total_timesteps | 65536    |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 98.7       |
|    ep_rew_mean          | -120       |
| time/                   |            |
|    fps                  | 1825       |
|    iterations           | 2          |
|    time_elapsed         | 71         |
|    total_timesteps      | 131072     |
| train/                  |            |
|    approx_kl            | 0.01231358 |
|    clip_fraction        | 0.132      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.38      |
|    explained_variance   | 0.00

In [36]:
eval_env = gym.make("LunarLander-v2")
mean_reward, std_reward =  evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")


/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward=-58.83 +/- 22.714343888064885


Proximal Policy Optimization big ideas
- compare two networks, a "current policy" and a "refinement" policy that collects samples
- we look at the ratio between the two polciies

In [25]:
## model 2: Deep Q Network, DQN
from stable_baselines3 import DQN

model2 = DQN("MlpPolicy",env, verbose=1)
model2.learn(total_timesteps=int(2e5))
model2_name = "base DQN model"
model2.save(model2_name)

Streaming output truncated to the last 5000 lines.
|    n_updates        | 66       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 79.3     |
|    ep_rew_mean      | -290     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 4920     |
|    time_elapsed     | 11       |
|    total_timesteps  | 58880    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.83     |
|    n_updates        | 69       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 78.3     |
|    ep_rew_mean      | -292     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 4922     |
|    time_elapsed     | 12       |
|    total_timesteps  | 59168    |
| tr

In [37]:
eval_env = gym.make("LunarLander-v2")
mean_reward2, std_reward2 =  evaluate_policy(model2, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")


/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward=-58.83 +/- 22.714343888064885


SQN big ideas
- replay experiences to learn from past policies
- clip/normalize the rewards 
- freeze the q-network


Without any hp tuning, it seems like the base ppo model performs better than DQN.

In [27]:
model2.policy

DQNPolicy(
  (q_net): QNetwork(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (q_net): Sequential(
      (0): Linear(in_features=8, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=4, bias=True)
    )
  )
  (q_net_target): QNetwork(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (q_net): Sequential(
      (0): Linear(in_features=8, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=4, bias=True)
    )
  )
)

So it seems like the DQN model has a Q-learned, deep network.

- Linear + ReLU activation

In [28]:
model2.learning_rate

0.0001

In [29]:
model2.batch_size

32

In [30]:
model2.tau

1.0

In [31]:
model2.gamma

0.99